In [2]:
## %%
# %pylab inline
# plot([1,2,3],[2,3,4])
# https://stackoverflow.com/questions/20309456/call-a-function-from-another-file

import pandas as pd
import numpy as np
import glob
import time
import datetime
import pathlib

pd.set_option('display.float_format', '{:.2f}'.format)


class FS:
    """
    FS = financial statement.
    Subject = BS, CF, or IF.
    {'df_table': fs.df_table(),
    'df_table_pct':fs.df_table_pct(),
    'chart_x_dates': chart_x_dates,
    'chart_y_revenue':chart_y_revenue,
    'df_json': df_json,
    'df_titles':df_titles}
    """

    def isnumber(x):
        try:
            float(x)
            return True
        except:
            return False
            
    pd.set_option('display.float_format', '{:.2f}'.format)

    def __init__(self,subject,symbol):
        if subject=="BS":
            subject="Balance Sheet"
        elif subject=="CF":
            subject="Cash Flow Statement"
        elif subject=="IS":
            subject="Income Statement"
        pd.set_option('display.float_format', '{:.2f}'.format)
        self.subject = subject
        self.symbol = symbol
        csv_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/{}/*_{}_*".format(self.subject,self.symbol))[-1] #.format("NLOK"))[-1]
        # csv_file = glob.glob("data\Historical Financial Statements\*\year\{}\*_{}_*".format(self.subject,self.symbol))[-1] #.format("NLOK"))[-1]
        self.year = pd.read_csv(csv_file)
        df = self.year[0:].iloc[::-1]
        df['Quarter & Year'] = df['period']+" "+(df['date'].astype(str).str[0:4])#((df_bs['date'].astype(str).str[0:4].astype(int))-1).astype(str)
        df = df.drop([ 'Unnamed: 0','symbol','fillingDate','period','link'],axis=1)
        df.columns = [
            'Date',
            'Date & Time Filing Accepted',
            'Revenue (Sales)',
            'Cost of Revenue (Sales)',
            'Gross Profit (Income)',
            'Gross Profit (Income) Ratio',
            'Research and Development (R&D) Expenses',
            'Sales, General and Administrative (SG&A) Expenses',
            'Selling and Marketing (S&M) Expenses',
            'Other Expenses',
            'Operating Expenses',
            'Cost and Expenses',
            'Interest Expense',
            'Depreciation and Amortization (D&A)',
            'EBITDA',
            'EBITDA Ratio',
            'Operating Income',
            'Operating Income Ratio',
            'Total Other Income Expenses (Net)',
            'Income Before Tax',
            'Income Before Tax Ratio',
            'Income Tax Expense',
            'Net Income',
            'Net Income Ratio',
            'EPS',
            'EPS Diluted',
            'Weighted Average Shares Outstanding',
            'Weighted Average Shares Outstanding (Diluted)',
            'SEC Filing', 'Quarter & Year'
                        ]

        cols = [ 'Date','Quarter & Year', 'Revenue (Sales)',
            'Cost of Revenue (Sales)',
            'Gross Profit (Income)',
            'Gross Profit (Income) Ratio',
            'Research and Development (R&D) Expenses',
            'Sales, General and Administrative (SG&A) Expenses',
            'Other Expenses',
            'Operating Expenses',
            'Cost and Expenses',
            'Interest Expense',
            'Depreciation and Amortization (D&A)',
            'EBITDA',
            'EBITDA Ratio',
            'Operating Income',
            'Operating Income Ratio',
            'Total Other Income Expenses (Net)',
            'Income Before Tax',
            'Income Before Tax Ratio',
            'Income Tax Expense',
            'Net Income',
            'Net Income Ratio',
            'EPS',
            'EPS Diluted',
            'Weighted Average Shares Outstanding',
            'Weighted Average Shares Outstanding (Diluted)',
            'SEC Filing','Date & Time Filing Accepted']
        self.df = df[cols]

    def isnumber(x):
        try:
            float(x)
            return True
        except:
            return False
            
    def first(self):
        earliest_year = list((self.year['date'].astype(str).str[0:4]))[-1]
        return(earliest_year)

    def last(self):
        latest_year = list((self.year['date'].astype(str).str[0:4]))[0]
        return(latest_year)  
    def df_table_pct(self):
        df_pct_chg = self.df
        pct_chg_cols = (self.df.select_dtypes(include=['number']).pct_change(-1))
        df_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
        df_pct_chg = df_str.join(pct_chg_cols)[list(self.df)]
        df_pct_chg_t = df_pct_chg.transpose()
        df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
        df_pct_chg_t = df_pct_chg_t.iloc[1:]

        df_pct_chg_t = df_pct_chg.transpose()
        df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
        df_pct_chg_t = df_pct_chg_t.iloc[1:]
        df_pct_chg_t['']=df_pct_chg_t.index
        df_pct_chg_t.index = range(len(df_pct_chg_t))

        cols = list(df_pct_chg_t.columns)
        cols = [cols[-1]] + cols[:-1]
        df_pct_chg_t = df_pct_chg_t[cols]


        df_t = self.df.transpose()
        df_t.columns = list(self.df['Quarter & Year'])
        df_t = df_t.iloc[1:]
        df_t['']=df_t.index
        df_t.index = range(len(df_t))

        cols = list(df_t.columns)
        cols = [cols[-1]] + cols[:-1]
        df_t = df_t[cols]

        df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')
        df_table_pct = [df_pct]
        return df_table_pct
    
    def df_table(self):
        fs = FS(self.subject,self.symbol)
        df_t = self.df.transpose()
        df_t.columns = list(self.df['Quarter & Year'])
        df_t = df_t.iloc[1:]
        df_t['']=df_t.index
        df_t.index = range(len(df_t))        
        df_t = df_t[df_t.columns[::-1]]
        cols = list(df_t.columns)
        cols = [cols[-1]] + cols[:-1]
        df_t = df_t[cols]
        # df_html = df_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable2"')# dt-responsive" id="df_myTable"')

        df_n = self.df[self.df.applymap(FS.isnumber)]
        df_n[df_n < 2] = np.nan
        # pd.DataFrame(df_n.sum())#axis=0))
        df_n_sum = pd.DataFrame(df_n.sum())
        df_n_sum[df_n_sum == 0] = ""
        new_header = df_n_sum.iloc[0] #grab the first row for the header
        df_n_sum = df_n_sum[1:] #take the data less the header row
        df_n_sum.columns = new_header #set the header row as the df header
        df_n_sum.index = range(len(df_n_sum))
        df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(fs.last(),fs.first()), 'Line Items'))
        df_t = df_t[0:25]

        col_list = []
        n=0
        while n<len(list(df_t))-0:
            if n<6:
                    col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
            else:
                col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
            col_list.append(col_item)
            n+=1
        col_list_str = ''.join(map(str, col_list))
        df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))


        df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
        df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
        df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
        df_html = df_html[0:]
        df_table = [df_html[0:]]
        return df_table

    def df_json(self):
        df = self.df
        df['Date'] = pd.to_datetime(df['Date']).values.astype(np.int64) // 10 ** 6
        df = df[['Date', 'Revenue (Sales)']].dropna().to_numpy().tolist()        
        return df

    def df_table_raw(self):
        return self.df

    def df_labels(self):
        df = self.df
        df['Date'] = pd.to_datetime(df['Date']).values.astype(np.int64) // 10 ** 6
        labels = df[['Date']].to_numpy().tolist(),
        # labels = list(df['Date'])#[0:19]
        # labels = list(df['Revenue (Sales)'])
        return labels

    def df_price(self):
        df = self.df
        values = list(df['Revenue (Sales)'])
        # values = list(df['Beginning Price'])[0:19]
        return values

    def df_values(self):
        fs = FS(self.subject,self.symbol)
        fs.df_table()
        # df = self.df[['date','revenue']].dropna() #.fillna(0)#.fillna(method='bfill')
        df = self.df
        df['Date'] = (pd.to_datetime(self.df['Date']).values.astype(np.int64) // 10 ** 6)
        df = df[['Date', 'Revenue (Sales)']]
        # df_json = df.to_numpy().tolist()
        df_json = fs.df_json()
        chart_x_dates =  df['Date'].to_list()
        chart_y_revenue = df['Revenue (Sales)'].to_list()
        df_titles = df.columns.values
        # df_json = fs.df_json()
        # {'df_json': df_json, 'chart_x_dates':chart_x_dates,'chart_y_revenue':chart_y_revenue,'df_titles':df_titles }
        return {'df_table': fs.df_table()
                , 'df_table_pct':fs.df_table_pct(),
               'chart_x_dates': chart_x_dates, 'chart_y_revenue':chart_y_revenue,
               'df_json': df_json, 'df_titles':df_titles}


# def FS_csv(subject, symbol):
#     csv_file = glob.glob("..\data\Historical Financial Statements\*\year\{}\*_{}_*".format(subject,symbol))[-1] #.format("NLOK"))[-1]
#     df = pd.read_csv(csv_file)
#     return(df)

# def FS_first_year(df):
#     earliest_year = list((df['date'].astype(str).str[0:4]))[-1]
#     return(earliest_year)

# def FS_latest_year(df):
#     latest_year = list((df_bs['date'].astype(str).str[0:4]))[0]
#     return(latest_year)

# print(FS_first_year(FS_csv("Income Statement","AAPL")))


# labels = list(FS("IS","AAPL")['Year'])[0:19]
# values = list(FS("IS","AAPL")['Beginning Price'])[0:19]

# tables=FS("IS","AAPL").df_values()['df_table'],

table_pct = FS("IS","AAPL").df_values()['df_table_pct'],
df_date = FS("IS","AAPL").df_values()['chart_x_dates'],
df_rev = FS("IS","AAPL").df_values()['chart_y_revenue'],
df_json = FS("IS","AAPL").df_values()['df_json'],
titles=FS("IS","AAPL").df_values()['df_titles']
print(str(df_json)[0:200])
# print(FS("IS","AAPL").df_table_raw())
print(FS("IS","AAPL").last())

subject="Income Statement"
symbol="AAPL"
csv_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/{}/*_{}_*".format(subject,symbol))[-1]
# csv_file = glob.glob("data\Historical Financial Statements\*\year\{}\*_{}_*".format(subject,symbol))[-1]
# csv_file = glob.glob("data\Historical Financial Statements\*\year\*")

print(csv_file)

print("up next")
# print(pd.read_csv(csv_file))
print(str(df_json)[0:200])
print((FS("IS","AAPL").df_price()))
print(( FS("IS","AAPL").df_labels()))


([[433728000000, 983000000], [465177600000, 1516000000], [496627200000, 1918000000], [528076800000, 1902000000], [559526400000, 2661000000], [591580800000, 4071000000], [623030400000, 5284000000], [65
2019
../Charts_TenDollarData/financial_statements/data/Historical Financial Statements\Large Cap+\year\Income Statement\Y_AAPL_income_statements.csv
up next
([[433728000000, 983000000], [465177600000, 1516000000], [496627200000, 1918000000], [528076800000, 1902000000], [559526400000, 2661000000], [591580800000, 4071000000], [623030400000, 5284000000], [65
[983000000, 1516000000, 1918000000, 1902000000, 2661000000, 4071000000, 5284000000, 5558000000, 6309000000, 7087000000, 7977000000, 9189000000, 11062000000, 9833000000, 7081000000, 5941000000, 6134000000, 7983000000, 5363000000, 5742000000, 6207000000, 8279000000, 13931000000, 19315000000, 24006000000, 32479000000, 42905000000, 65225000000, 108249000000, 156508000000, 170910000000, 182795000000, 233715000000, 215639000000, 229234000000, 

In [23]:
subject = "Technology"
subject = 'Income Statement'
subject = 'Balance Sheet'


In [27]:
csv_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/{}/*~{}~*".format(subject,symbol))[-1] #.format("NLOK"))[-1]
        # csv_file = glob.glob("data\Historical Financial Statements\*\year\{}\*_{}_*".format(self.subject,self.symbol))[-1] #.format("NLOK"))[-1]
df = pd.read_csv(csv_file)

In [38]:
del df

In [40]:
csv_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/Balance Sheet/*~{}~*".format(symbol))[-1] #.format("NLOK"))[-1]
        # csv_file = glob.glob("data\Historical Financial Statements\*\year\{}\*_{}_*".format(self.subject,self.symbol))[-1] #.format("NLOK"))[-1]
df = pd.read_csv(csv_file)
df

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,...,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,totalInvestments,totalDebt,netDebt,link,finalLink
0,0,2020-09-26,AAPL,2020-10-30,2020-10-29 18:06:25,FY,38016000000,52927000000,90943000000,16120000000,...,14966000000,-406000000.00,33802237000.00,65339000000,323888000000,153814000000.00,107440000000.00,69424000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,0,2019-09-28,AAPL,2019-10-31 00:00:00,2019-10-30 18:12:36,FY,48844000000,51713000000,100557000000,45804000000,...,45898000000,-584000000.00,-1291000000.00,90488000000,338516000000,157054000000.00,108047000000.00,59203000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,0,2018-09-29,AAPL,2018-11-05 00:00:00,2018-11-05 08:01:40,FY,25913000000,40388000000,66301000000,48995000000,...,70400000000,-3454000000.00,-245000000.00,107147000000,365725000000,211187000000.00,114483000000.00,88570000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,0,2017-09-30,AAPL,2017-11-03 00:00:00,2017-11-03 08:01:37,FY,20289000000,53892000000,74181000000,35673000000,...,98330000000,-41293000000.00,0.00,134047000000,241272000000,0.00,115680000000.00,95391000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,0,2016-09-24,AAPL,2016-10-26 00:00:00,2016-10-26 16:42:16,FY,20484000000,46671000000,67155000000,29299000000,...,96364000000,634000000.00,-540000000.00,128249000000,321686000000,217101000000.00,87032000000.00,66548000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
5,0,2015-09-26,AAPL,2015-10-28 00:00:00,2015-10-28 16:31:09,FY,21120000000,20481000000,41601000000,30343000000,...,92284000000,-345000000.00,308000000.00,119355000000,290345000000,184546000000.00,64328000000.00,43208000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
6,0,2014-09-27,AAPL,2014-10-27 00:00:00,2014-10-27 17:11:55,FY,13844000000,11233000000,25077000000,27219000000,...,87152000000,1082000000.00,1324000000.00,111547000000,231839000000,141395000000.00,35295000000.00,21451000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
7,0,2013-09-28,AAPL,2013-10-30 00:00:00,2013-10-29 20:38:28,FY,14259000000,26287000000,40546000000,20641000000,...,104256000000,-471000000.00,-366000000.00,123549000000,207000000000,132502000000.00,16960000000.00,2701000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
8,0,2012-09-29,AAPL,2012-10-31 00:00:00,2012-10-31 17:07:19,FY,10746000000,18383000000,29129000000,18692000000,...,101289000000,499000000.00,491000000.00,118210000000,176064000000,110505000000.00,0.00,-10746000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
9,0,2011-09-24,AAPL,2011-10-26 00:00:00,2011-10-26 16:35:25,FY,9815000000,16137000000,25952000000,11717000000,...,62841000000,443000000.00,420000000.00,76615000000,116371000000,71755000000.00,0.00,-9815000000.00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...


In [95]:
cf_path = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/Cash Flow Statement/*~{}~*".format(symbol))[-1] #.format("NLOK"))[-1]
        # csv_file = glob.glob("data\Historical Financial Statements\*\year\{}\*_{}_*".format(self.subject,self.symbol))[-1] #.format("NLOK"))[-1]
cf_df = pd.read_csv(cf_path)
cf_df

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,netIncome,depreciationAndAmortization,deferredIncomeTax,stockBasedCompensation,...,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink
0,0,2020-09-26,AAPL,2020-10-30,2020-10-29 18:06:25,FY,57411000000,11056000000,-215000000.00,6829000000.00,...,-86820000000.00,0.00,-10435000000.00,39789000000,50224000000,80674000000,7309000000,73365000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,0,2019-09-28,AAPL,2019-10-31 00:00:00,2019-10-30 18:12:36,FY,55256000000,12547000000,-340000000.00,6068000000.00,...,-90976000000.00,0.00,24311000000.00,50224000000,25913000000,69391000000,-10495000000,58896000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,0,2018-09-29,AAPL,2018-11-05 00:00:00,2018-11-05 08:01:40,FY,59531000000,10903000000,-32590000000.00,5340000000.00,...,-87876000000.00,0.00,5624000000.00,25913000000,20289000000,77434000000,-13313000000,64121000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,0,2017-09-30,AAPL,2017-11-03 00:00:00,2017-11-03 08:01:37,FY,48351000000,10157000000,5966000000.00,4840000000.00,...,0.00,0.00,0.00,20289000000,20484000000,63598000000,-12795000000,50803000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,0,2016-09-24,AAPL,2016-10-26 00:00:00,2016-10-26 16:42:16,FY,45687000000,10505000000,4938000000.00,4210000000.00,...,-20890000000.00,0.00,-636000000.00,20484000000,21120000000,66231000000,-12734000000,53497000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
5,0,2015-09-26,AAPL,2015-10-28 00:00:00,2015-10-28 16:31:09,FY,53394000000,11257000000,1382000000.00,3586000000.00,...,-17716000000.00,0.00,7276000000.00,21120000000,13844000000,81266000000,-11488000000,69778000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
6,0,2014-09-27,AAPL,2014-10-27 00:00:00,2014-10-27 17:11:55,FY,39510000000,7946000000,2347000000.00,2863000000.00,...,-37549000000.00,0.00,-415000000.00,13844000000,14259000000,59713000000,-9813000000,49900000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
7,0,2013-09-28,AAPL,2013-10-30 00:00:00,2013-10-29 20:38:28,FY,37037000000,6757000000,1141000000.00,2253000000.00,...,-16379000000.00,0.00,3513000000.00,14259000000,10746000000,53666000000,-9076000000,44590000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
8,0,2012-09-29,AAPL,2012-10-31 00:00:00,2012-10-31 17:07:19,FY,41733000000,3277000000,4405000000.00,1740000000.00,...,-1698000000.00,0.00,931000000.00,10746000000,9815000000,50856000000,-9402000000,41454000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
9,0,2011-09-24,AAPL,2011-10-26 00:00:00,2011-10-26 16:35:25,FY,25922000000,1814000000,2868000000.00,1168000000.00,...,1444000000.00,0.00,-1446000000.00,9815000000,11261000000,37529000000,-7452000000,30077000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...


In [96]:
list(cf_df)

['Unnamed: 0',
 'date',
 'symbol',
 'fillingDate',
 'acceptedDate',
 'period',
 'netIncome',
 'depreciationAndAmortization',
 'deferredIncomeTax',
 'stockBasedCompensation',
 'changeInWorkingCapital',
 'accountsReceivables',
 'inventory',
 'accountsPayables',
 'otherWorkingCapital',
 'otherNonCashItems',
 'netCashProvidedByOperatingActivities',
 'investmentsInPropertyPlantAndEquipment',
 'acquisitionsNet',
 'purchasesOfInvestments',
 'salesMaturitiesOfInvestments',
 'otherInvestingActivites',
 'netCashUsedForInvestingActivites',
 'debtRepayment',
 'commonStockIssued',
 'commonStockRepurchased',
 'dividendsPaid',
 'otherFinancingActivites',
 'netCashUsedProvidedByFinancingActivities',
 'effectOfForexChangesOnCash',
 'netChangeInCash',
 'cashAtEndOfPeriod',
 'cashAtBeginningOfPeriod',
 'operatingCashFlow',
 'capitalExpenditure',
 'freeCashFlow',
 'link',
 'finalLink']

In [152]:
df = df[0:].iloc[::-1]#.dropna()
df_bs = df
#region Pandas data manipulation
# df_bs['Quarter & Year'] = df_bs['period']+" "+(df_bs['date'].astype(str).str[0:4])#((df_bs['date'].astype(str).str[0:4].astype(int))-1).astype(str)
df_bs['Quarter & Year'] = (df_bs['date'].astype(str).str[0:4]).astype(int)#((df_bs['date'].astype(str).str[0:4].astype(int))-1).astype(str)

latest_year = list((df_bs['date'].astype(str).str[0:4]))[0]
earliest_year = list((df_bs['date'].astype(str).str[0:4]))[-1]

df_bs = df_bs.drop([ 'Unnamed: 0','date',
 'symbol',
 'fillingDate','period','link','acceptedDate'],axis=1)

In [153]:
df_bs.columns = [
    'Cash and Cash Equivalents',
    'Short Term Investments',
    'Cash And Short-Term Investments',
    'Net Receivables',
    'Inventory',
    'Other Current Assets',
    'Total Current Assets',
    'PP&E',
    'Goodwill',
    'Intangible Assets',
    'Goodwill and Intangible Assets',
    'Long-term Investments',
    'Tax Assets',
    'Other Non-Current Assets',
    'Total Non-Current Assets',
    'Other Assets',
    'TotalAssets',
    'Accounts Payables',
    'Short-term Debt',
    'Tax Payables',
    'Deferred Revenue',
    'Other Current Liabilities',
    'Total Current Liabilities',
    'Long-term Debt',
    'Deferred Revenue Non-Current',
    'Deferred Tax Liabilities Non-Current',
    'Other Non-Current Liabilities',
    'Total Non-Current Liabilities',
    'Other Liabilities',
    'Total Liabilities',
    'Common Stock',
    'Retained Earnings',
    'Accumulated Other Comprehensive Income Loss',
    'Other Total Stockholders Equity',
    'Total Stockholders Equity',
    'Total Liabilities & Stockholders Equity',
    'Total Investments',
    'Total Debt',
    'Net Debt',
    'finalLink','Quarter & Year']

In [154]:
cols = ['Quarter & Year',
        'Cash and Cash Equivalents',
        'Short Term Investments',
        'Cash And Short-Term Investments',
        'Net Receivables',
        'Inventory',
        'Other Current Assets',
        'Total Current Assets',
        'PP&E',
        'Goodwill',
        'Intangible Assets',
        'Goodwill and Intangible Assets',
        'Long-term Investments',
        'Tax Assets',
        'Other Non-Current Assets',
        'Total Non-Current Assets',
        'Other Assets',
        'TotalAssets',
        'Accounts Payables',
        'Short-term Debt',
        'Tax Payables',
        'Deferred Revenue',
        'Other Current Liabilities',
        'Total Current Liabilities',
        'Long-term Debt',
        'Deferred Revenue Non-Current',
        'Deferred Tax Liabilities Non-Current',
        'Other Non-Current Liabilities',
        'Total Non-Current Liabilities',
        'Other Liabilities',
        'Total Liabilities',
        'Common Stock',
        'Retained Earnings',
        'Accumulated Other Comprehensive Income Loss',
        'Other Total Stockholders Equity',
        'Total Stockholders Equity',
        'Total Liabilities & Stockholders Equity',
        'Total Investments',
        'Total Debt',
        'Net Debt',
        'finalLink']

df_bs = df_bs[cols]
df_bs

,Quarter & Year,Cash and Cash Equivalents,Short Term Investments,Cash And Short-Term Investments,Net Receivables,Inventory,Other Current Assets,Total Current Assets,PP&E,Goodwill,...,Common Stock,Retained Earnings,Accumulated Other Comprehensive Income Loss,Other Total Stockholders Equity,Total Stockholders Equity,Total Liabilities & Stockholders Equity,Total Investments,Total Debt,Net Debt,finalLink
0,2020,38016000000,52927000000,90943000000,16120000000,4061000000,32589000000,143713000000,36766000000,0.00,...,16976763000,14966000000,-406000000.00,33802237000.00,65339000000,323888000000,153814000000.00,107440000000.00,69424000000.00,https://www.sec.gov/Archives/edgar/data/320193...
1,2019,48844000000,51713000000,100557000000,45804000000,4106000000,12352000000,162819000000,37378000000,0.00,...,45174000000,45898000000,-584000000.00,-1291000000.00,90488000000,338516000000,157054000000.00,108047000000.00,59203000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2,2018,25913000000,40388000000,66301000000,48995000000,3956000000,12087000000,131339000000,41304000000,0.00,...,40201000000,70400000000,-3454000000.00,-245000000.00,107147000000,365725000000,211187000000.00,114483000000.00,88570000000.00,https://www.sec.gov/Archives/edgar/data/320193...
3,2017,20289000000,53892000000,74181000000,35673000000,4855000000,13936000000,128645000000,33783000000,5717000000.00,...,35867000000,98330000000,-41293000000.00,0.00,134047000000,241272000000,0.00,115680000000.00,95391000000.00,https://www.sec.gov/Archives/edgar/data/320193...
4,2016,20484000000,46671000000,67155000000,29299000000,2132000000,8283000000,106869000000,27010000000,5414000000.00,...,31251000000,96364000000,634000000.00,-540000000.00,128249000000,321686000000,217101000000.00,87032000000.00,66548000000.00,https://www.sec.gov/Archives/edgar/data/320193...
5,2015,21120000000,20481000000,41601000000,30343000000,2349000000,15085000000,89378000000,22471000000,5116000000.00,...,27416000000,92284000000,-345000000.00,308000000.00,119355000000,290345000000,184546000000.00,64328000000.00,43208000000.00,https://www.sec.gov/Archives/edgar/data/320193...
6,2014,13844000000,11233000000,25077000000,27219000000,2111000000,14124000000,68531000000,20624000000,4616000000.00,...,23313000000,87152000000,1082000000.00,1324000000.00,111547000000,231839000000,141395000000.00,35295000000.00,21451000000.00,https://www.sec.gov/Archives/edgar/data/320193...
7,2013,14259000000,26287000000,40546000000,20641000000,1764000000,10335000000,73286000000,16597000000,1577000000.00,...,19764000000,104256000000,-471000000.00,-366000000.00,123549000000,207000000000,132502000000.00,16960000000.00,2701000000.00,https://www.sec.gov/Archives/edgar/data/320193...
8,2012,10746000000,18383000000,29129000000,18692000000,791000000,9041000000,57653000000,15452000000,1135000000.00,...,16422000000,101289000000,499000000.00,491000000.00,118210000000,176064000000,110505000000.00,0.00,-10746000000.00,https://www.sec.gov/Archives/edgar/data/320193...
9,2011,9815000000,16137000000,25952000000,11717000000,776000000,6543000000,44988000000,7777000000,896000000.00,...,13331000000,62841000000,443000000.00,420000000.00,76615000000,116371000000,71755000000.00,0.00,-9815000000.00,https://www.sec.gov/Archives/edgar/data/320193...


In [162]:

urilist = ["/log", "cash_and_cash_equivalents","/current_ratio", "/loginexample"]
url_is_list = [ 'revenue_sales',
            'cost_of_sales_cost_of_revenue_cost_of_goods_sold',
            'gross_profit_gross_income',
            'gross_income_ratio_gross_profit_ratio',
            'r%26d_research_and_development_expenses',
            'sg%26a_sga_sales_general_and_administrative_expenses',
            'other_expenses',
            'operating_expenses',
            '',
            'interest_expense',
            'd%26a_depreciation_and_amortization',
            'ebitda',
            'ebitda_ratio',
            'operating_income',
            'operating_income_ratio',
            'other_income_other_expenses_net',
            'income_before_tax_provisions',
            'income_before_tax_ratio',
            'income_tax_expense_provisions',
            'net_income',
            'net_income_ratio',
            'eps_earnings_per_share',
            'eps_earnings_per_share_diluted',
            'shares_outstanding_for_eps',
            'shares_outstanding_weighted_for_eps',]
url_bs_list = ['quarter_and_year','cash_and_cash_equivalents',
            'short_term_investments',
            'cash_and_short_term_investments',
            'net_receivables',
            'inventory',
            'other_current_assets',
            'total_current_assets',
            'pp_and_e',
            'goodwill',
            'intangible_assets',
            'goodwill_and_intangible_assets',
            'long_term_investments',
            'tax_assets',
            'other_non_current_assets',
            'total_non_current_assets',
            'other_assets',
            'total_assets',
            'accounts_payables',
            'short_term_debt',
            'tax_payables',
            'deferred_revenue',
            'other_current_liabilities',
            'total_current_liabilities',
            'long_term_debt',
            'deferred_revenue_non_current',
            'deferred_tax_liabilities_non_current',
            'other_non_current_liabilities',
            'total_non_current_liabilities',
            'other_liabilities',
            'total_liabilities',
            'common_stock',
            'retained_earnings',
            'accumulated_other_comprehensive_income_loss',
            'other_total_stockholders_equity',
            'total_stockholders_equity',
            'total_liabilities_and_stockholders_equity',
            'total_investments',
            'total_debt',
            'net_debt','final_link']
url_cf_list = ['net_income',
                'depreciation_and_amortization',
                'deferred_income_tax',
                'stock_based_compensation',
                'change_in_working_capital',
                'accounts_receivable',
                'inventory',
                'accounts_payables',
                'other_working_capital',
                'other_non_cash_items',
                'net_cash_provided_by_operating_activities',
                'investments_in_property_plant_and_equipment',
                'acquisitions_net',
                'purchases_of_investments',
                'sales_maturities_of_investments',
                'other_investing_activities',
                'net_cash_used_for_investing_activities',
                'debt_repayment',
                'common_stock_issued',
                'common_stock_repurchased',
                'dividends_paid',
                'other_financing_activities',
                'net_cash_used_provided_by_financing_activities',
                'effect_of_forex_changes_on_cash',
                'net_change_in_cash',
                'cash_at_end_of_period',
                'cash_at_beginning_of_period',
                'operating_cash_flow',
                'capital_expenditure',
                'free_cash_flow']
if "{}".format(urilist) in url_is_list:
    fin_statement_dir = "Income Statement"
elif "{}".format(urilist) in url_bs_list:
    fin_metric_pos = url_bs_list.index("{}".format(urilist))
    fin_statement_dir = "Balance Sheet"
    fin_statement_cols = ['Cash and Cash Equivalents',
                        'Short Term Investments',
                        'Cash And Short-Term Investments',
                        'Net Receivables',
                        'Inventory',
                        'Other Current Assets',
                        'Total Current Assets',
                        'PP&E',
                        'Goodwill',
                        'Intangible Assets',
                        'Goodwill and Intangible Assets',
                        'Long-term Investments',
                        'Tax Assets',
                        'Other Non-Current Assets',
                        'Total Non-Current Assets',
                        'Other Assets',
                        'TotalAssets',
                        'Accounts Payables',
                        'Short-term Debt',
                        'Tax Payables',
                        'Deferred Revenue',
                        'Other Current Liabilities',
                        'Total Current Liabilities',
                        'Long-term Debt',
                        'Deferred Revenue Non-Current',
                        'Deferred Tax Liabilities Non-Current',
                        'Other Non-Current Liabilities',
                        'Total Non-Current Liabilities',
                        'Other Liabilities',
                        'Total Liabilities',
                        'Common Stock',
                        'Retained Earnings',
                        'Accumulated Other Comprehensive Income Loss',
                        'Other Total Stockholders Equity',
                        'Total Stockholders Equity',
                        'Total Liabilities & Stockholders Equity',
                        'Total Investments',
                        'Total Debt',
                        'Net Debt',
                        'finalLink','Quarter & Year']
    cols = ['Quarter & Year',
            'Cash and Cash Equivalents',
            'Short Term Investments',
            'Cash And Short-Term Investments',
            'Net Receivables',
            'Inventory',
            'Other Current Assets',
            'Total Current Assets',
            'PP&E',
            'Goodwill',
            'Intangible Assets',
            'Goodwill and Intangible Assets',
            'Long-term Investments',
            'Tax Assets',
            'Other Non-Current Assets',
            'Total Non-Current Assets',
            'Other Assets',
            'TotalAssets',
            'Accounts Payables',
            'Short-term Debt',
            'Tax Payables',
            'Deferred Revenue',
            'Other Current Liabilities',
            'Total Current Liabilities',
            'Long-term Debt',
            'Deferred Revenue Non-Current',
            'Deferred Tax Liabilities Non-Current',
            'Other Non-Current Liabilities',
            'Total Non-Current Liabilities',
            'Other Liabilities',
            'Total Liabilities',
            'Common Stock',
            'Retained Earnings',
            'Accumulated Other Comprehensive Income Loss',
            'Other Total Stockholders Equity',
            'Total Stockholders Equity',
            'Total Liabilities & Stockholders Equity',
            'Total Investments',
            'Total Debt',
            'Net Debt',
            'finalLink']        
    fin_metric_name = fin_statement_cols[fin_metric_pos-1]

elif "{}".format(urilist) in url_cf_list:
    fin_statement_dir = "Cash Flow Statement"

else:
    pass

In [163]:
df_bs_url = df_bs
df_bs_url.columns = ['quarter_and_year','cash_and_cash_equivalents',
                    'short_term_investments',
                    'cash_and_short_term_investments',
                    'net_receivables',
                    'inventory',
                    'other_current_assets',
                    'total_current_assets',
                    'pp_and_e',
                    'goodwill',
                    'intangible_assets',
                    'goodwill_and_intangible_assets',
                    'long_term_investments',
                    'tax_assets',
                    'other_non_current_assets',
                    'total_non_current_assets',
                    'other_assets',
                    'total_assets',
                    'accounts_payables',
                    'short_term_debt',
                    'tax_payables',
                    'deferred_revenue',
                    'other_current_liabilities',
                    'total_current_liabilities',
                    'long_term_debt',
                    'deferred_revenue_non_current',
                    'deferred_tax_liabilities_non_current',
                    'other_non_current_liabilities',
                    'total_non_current_liabilities',
                    'other_liabilities',
                    'total_liabilities',
                    'common_stock',
                    'retained_earnings',
                    'accumulated_other_comprehensive_income_loss',
                    'other_total_stockholders_equity',
                    'total_stockholders_equity',
                    'total_liabilities_and_stockholders_equity',
                    'total_investments',
                    'total_debt',
                    'net_debt','final_link']

In [164]:
df_bs.index = df_bs['quarter_and_year']
df_bs_html = df_bs[['cash_and_cash_equivalents']].transpose().to_html()
df_bs_html.replace("'","")

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>quarter_and_year</th>\n      <th>2020</th>\n      <th>2019</th>\n      <th>2018</th>\n      <th>2017</th>\n      <th>2016</th>\n      <th>2015</th>\n      <th>2014</th>\n      <th>2013</th>\n      <th>2012</th>\n      <th>2011</th>\n      <th>2010</th>\n      <th>2009</th>\n      <th>2008</th>\n      <th>2007</th>\n      <th>2006</th>\n      <th>2005</th>\n      <th>2004</th>\n      <th>2003</th>\n      <th>2002</th>\n      <th>2001</th>\n      <th>2000</th>\n      <th>1999</th>\n      <th>1998</th>\n      <th>1997</th>\n      <th>1996</th>\n      <th>1995</th>\n      <th>1994</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>cash_and_cash_equivalents</th>\n      <td>38016000000</td>\n      <td>48844000000</td>\n      <td>25913000000</td>\n      <td>20289000000</td>\n      <td>20484000000</td>\n      <td>21120000000</td>\n      <td>13844000000</td>\n      <td>14259000000</td>\n   

In [165]:
df_bs_pct_chg = df_bs
pct_chg_cols = (df_bs.select_dtypes(include=['number']).pct_change(-1))
df_bs_str = df_bs_pct_chg.drop(list(pct_chg_cols), axis=1)
df_bs_pct_chg = df_bs_str.join(pct_chg_cols)[list(df_bs)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_bs_pct_chg_t = df_bs_pct_chg.transpose()
# df_bs_pct_chg_t.columns = list(df_bs_pct_chg['Quarter & Year'])
# df_bs_pct_chg_t = df_bs_pct_chg_t.iloc[1:]

In [166]:
fin_metric_history.round(decimals=2)

quarter_and_year
FY 2020    38016000000
FY 2019    48844000000
FY 2018    25913000000
FY 2017    20289000000
FY 2016    20484000000
FY 2015    21120000000
FY 2014    13844000000
FY 2013    14259000000
FY 2012    10746000000
FY 2011     9815000000
FY 2010    11261000000
FY 2009     5263000000
FY 2008    11875000000
FY 2007     9352000000
FY 2006     6392000000
FY 2005     3491000000
FY 2004     2969000000
FY 2003     3396000000
FY 2002     2252000000
FY 2001     2310000000
FY 2000     1191000000
FY 1999     1326000000
FY 1998     1481000000
FY 1997     1230000000
FY 1996     1552000000
FY 1995      756000000
FY 1994     1203488000
Name: cash_and_cash_equivalents, dtype: int64

In [167]:
df_bs

,quarter_and_year,cash_and_cash_equivalents,short_term_investments,cash_and_short_term_investments,net_receivables,inventory,other_current_assets,total_current_assets,pp_and_e,goodwill,...,common_stock,retained_earnings,accumulated_other_comprehensive_income_loss,other_total_stockholders_equity,total_stockholders_equity,total_liabilities_and_stockholders_equity,total_investments,total_debt,net_debt,final_link
quarter_and_year,,,,,,,,,,,,,,,,,,,,,
2020,2020,38016000000,52927000000,90943000000,16120000000,4061000000,32589000000,143713000000,36766000000,nan,...,16976763000,14966000000,nan,33802237000.00,65339000000,323888000000,153814000000.00,107440000000.00,69424000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2019,2019,48844000000,51713000000,100557000000,45804000000,4106000000,12352000000,162819000000,37378000000,nan,...,45174000000,45898000000,nan,nan,90488000000,338516000000,157054000000.00,108047000000.00,59203000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2018,2018,25913000000,40388000000,66301000000,48995000000,3956000000,12087000000,131339000000,41304000000,nan,...,40201000000,70400000000,nan,nan,107147000000,365725000000,211187000000.00,114483000000.00,88570000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2017,2017,20289000000,53892000000,74181000000,35673000000,4855000000,13936000000,128645000000,33783000000,5717000000.00,...,35867000000,98330000000,nan,nan,134047000000,241272000000,nan,115680000000.00,95391000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2016,2016,20484000000,46671000000,67155000000,29299000000,2132000000,8283000000,106869000000,27010000000,5414000000.00,...,31251000000,96364000000,634000000.00,nan,128249000000,321686000000,217101000000.00,87032000000.00,66548000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2015,2015,21120000000,20481000000,41601000000,30343000000,2349000000,15085000000,89378000000,22471000000,5116000000.00,...,27416000000,92284000000,nan,308000000.00,119355000000,290345000000,184546000000.00,64328000000.00,43208000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2014,2014,13844000000,11233000000,25077000000,27219000000,2111000000,14124000000,68531000000,20624000000,4616000000.00,...,23313000000,87152000000,1082000000.00,1324000000.00,111547000000,231839000000,141395000000.00,35295000000.00,21451000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2013,2013,14259000000,26287000000,40546000000,20641000000,1764000000,10335000000,73286000000,16597000000,1577000000.00,...,19764000000,104256000000,nan,nan,123549000000,207000000000,132502000000.00,16960000000.00,2701000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2012,2012,10746000000,18383000000,29129000000,18692000000,791000000,9041000000,57653000000,15452000000,1135000000.00,...,16422000000,101289000000,499000000.00,491000000.00,118210000000,176064000000,110505000000.00,nan,nan,https://www.sec.gov/Archives/edgar/data/320193...


In [168]:


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_bs_n = df_bs[df_bs.applymap(isnumber)]
df_bs_n[df_bs_n < 2] = np.nan

In [169]:
df_bs_n

,quarter_and_year,cash_and_cash_equivalents,short_term_investments,cash_and_short_term_investments,net_receivables,inventory,other_current_assets,total_current_assets,pp_and_e,goodwill,...,common_stock,retained_earnings,accumulated_other_comprehensive_income_loss,other_total_stockholders_equity,total_stockholders_equity,total_liabilities_and_stockholders_equity,total_investments,total_debt,net_debt,final_link
quarter_and_year,,,,,,,,,,,,,,,,,,,,,
2020,2020,38016000000,52927000000,90943000000,16120000000,4061000000,32589000000,143713000000,36766000000,nan,...,16976763000,14966000000,nan,33802237000.00,65339000000,323888000000,153814000000.00,107440000000.00,69424000000.00,NaN
2019,2019,48844000000,51713000000,100557000000,45804000000,4106000000,12352000000,162819000000,37378000000,nan,...,45174000000,45898000000,nan,nan,90488000000,338516000000,157054000000.00,108047000000.00,59203000000.00,NaN
2018,2018,25913000000,40388000000,66301000000,48995000000,3956000000,12087000000,131339000000,41304000000,nan,...,40201000000,70400000000,nan,nan,107147000000,365725000000,211187000000.00,114483000000.00,88570000000.00,NaN
2017,2017,20289000000,53892000000,74181000000,35673000000,4855000000,13936000000,128645000000,33783000000,5717000000.00,...,35867000000,98330000000,nan,nan,134047000000,241272000000,nan,115680000000.00,95391000000.00,NaN
2016,2016,20484000000,46671000000,67155000000,29299000000,2132000000,8283000000,106869000000,27010000000,5414000000.00,...,31251000000,96364000000,634000000.00,nan,128249000000,321686000000,217101000000.00,87032000000.00,66548000000.00,NaN
2015,2015,21120000000,20481000000,41601000000,30343000000,2349000000,15085000000,89378000000,22471000000,5116000000.00,...,27416000000,92284000000,nan,308000000.00,119355000000,290345000000,184546000000.00,64328000000.00,43208000000.00,NaN
2014,2014,13844000000,11233000000,25077000000,27219000000,2111000000,14124000000,68531000000,20624000000,4616000000.00,...,23313000000,87152000000,1082000000.00,1324000000.00,111547000000,231839000000,141395000000.00,35295000000.00,21451000000.00,NaN
2013,2013,14259000000,26287000000,40546000000,20641000000,1764000000,10335000000,73286000000,16597000000,1577000000.00,...,19764000000,104256000000,nan,nan,123549000000,207000000000,132502000000.00,16960000000.00,2701000000.00,NaN
2012,2012,10746000000,18383000000,29129000000,18692000000,791000000,9041000000,57653000000,15452000000,1135000000.00,...,16422000000,101289000000,499000000.00,491000000.00,118210000000,176064000000,110505000000.00,nan,nan,NaN


In [171]:
df_bs

,quarter_and_year,cash_and_cash_equivalents,short_term_investments,cash_and_short_term_investments,net_receivables,inventory,other_current_assets,total_current_assets,pp_and_e,goodwill,...,common_stock,retained_earnings,accumulated_other_comprehensive_income_loss,other_total_stockholders_equity,total_stockholders_equity,total_liabilities_and_stockholders_equity,total_investments,total_debt,net_debt,final_link
quarter_and_year,,,,,,,,,,,,,,,,,,,,,
2020,2020,38016000000,52927000000,90943000000,16120000000,4061000000,32589000000,143713000000,36766000000,nan,...,16976763000,14966000000,nan,33802237000.00,65339000000,323888000000,153814000000.00,107440000000.00,69424000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2019,2019,48844000000,51713000000,100557000000,45804000000,4106000000,12352000000,162819000000,37378000000,nan,...,45174000000,45898000000,nan,nan,90488000000,338516000000,157054000000.00,108047000000.00,59203000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2018,2018,25913000000,40388000000,66301000000,48995000000,3956000000,12087000000,131339000000,41304000000,nan,...,40201000000,70400000000,nan,nan,107147000000,365725000000,211187000000.00,114483000000.00,88570000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2017,2017,20289000000,53892000000,74181000000,35673000000,4855000000,13936000000,128645000000,33783000000,5717000000.00,...,35867000000,98330000000,nan,nan,134047000000,241272000000,nan,115680000000.00,95391000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2016,2016,20484000000,46671000000,67155000000,29299000000,2132000000,8283000000,106869000000,27010000000,5414000000.00,...,31251000000,96364000000,634000000.00,nan,128249000000,321686000000,217101000000.00,87032000000.00,66548000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2015,2015,21120000000,20481000000,41601000000,30343000000,2349000000,15085000000,89378000000,22471000000,5116000000.00,...,27416000000,92284000000,nan,308000000.00,119355000000,290345000000,184546000000.00,64328000000.00,43208000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2014,2014,13844000000,11233000000,25077000000,27219000000,2111000000,14124000000,68531000000,20624000000,4616000000.00,...,23313000000,87152000000,1082000000.00,1324000000.00,111547000000,231839000000,141395000000.00,35295000000.00,21451000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2013,2013,14259000000,26287000000,40546000000,20641000000,1764000000,10335000000,73286000000,16597000000,1577000000.00,...,19764000000,104256000000,nan,nan,123549000000,207000000000,132502000000.00,16960000000.00,2701000000.00,https://www.sec.gov/Archives/edgar/data/320193...
2012,2012,10746000000,18383000000,29129000000,18692000000,791000000,9041000000,57653000000,15452000000,1135000000.00,...,16422000000,101289000000,499000000.00,491000000.00,118210000000,176064000000,110505000000.00,nan,nan,https://www.sec.gov/Archives/edgar/data/320193...


In [177]:
list(fin_metric_history)[0]

38016000000

In [178]:
million = 1000000
billion = 1000000000
fin_metric_history = df_bs['cash_and_cash_equivalents']
if list(fin_metric_history)[0]  > billion:
    print((df_bs['cash_and_cash_equivalents']/billion).round(decimals=2))
elif fin_metric_history[0]  > million:
    pass
#     "{:.2f}".format(value)
#     print(fin_metric_latest/billion)

quarter_and_year
2020   38.02
2019   48.84
2018   25.91
2017   20.29
2016   20.48
2015   21.12
2014   13.84
2013   14.26
2012   10.75
2011    9.82
2010   11.26
2009    5.26
2008   11.88
2007    9.35
2006    6.39
2005    3.49
2004    2.97
2003    3.40
2002    2.25
2001    2.31
2000    1.19
1999    1.33
1998    1.48
1997    1.23
1996    1.55
1995    0.76
1994    1.20
Name: cash_and_cash_equivalents, dtype: float64


In [132]:
((df_bs[['cash_and_cash_equivalents']]/billion).round(decimals=2))

,cash_and_cash_equivalents
quarter_and_year,
FY 2020,38.02
FY 2019,48.84
FY 2018,25.91
FY 2017,20.29
FY 2016,20.48
FY 2015,21.12
FY 2014,13.84
FY 2013,14.26
FY 2012,10.75


In [ ]:
cols = [ 'Quarter & Year', 'Revenue (Sales)',
 'Cost of Revenue (Sales)',
 'Gross Profit (Income)',
 'Gross Profit (Income) Ratio',
 'Research and Development (R&D) Expenses',
 'Sales, General and Administrative (SG&A) Expenses',
 'Other Expenses',
 'Operating Expenses',
 'Cost and Expenses',
 'Interest Expense',
 'Depreciation and Amortization (D&A)',
 'EBITDA',
 'EBITDA Ratio',
 'Operating Income',
 'Operating Income Ratio',
 'Total Other Income Expenses (Net)',
 'Income Before Tax',
 'Income Before Tax Ratio',
 'Income Tax Expense',
 'Net Income',
 'Net Income Ratio',
 'EPS',
 'EPS Diluted',
 'Weighted Average Shares Outstanding',
 'Weighted Average Shares Outstanding (Diluted)',
 'SEC Filing','Date & Time Filing Accepted']

In [33]:
current_assets = [s for s in list(df) if "totalCurrentAssets" in s]


In [35]:
df['totalCurrentAssets']/df['totalNonCurrentLiabilities']

0     0.94
1     1.14
2     0.92
3     0.92
4     0.93
5     0.99
6     1.21
7     1.84
8     2.99
9     3.82
10    6.25
11    7.25
12    7.80
13   14.48
14   19.35
15   17.14
16   24.00
17   25.05
18    9.89
19    8.82
20    7.11
21    8.44
22    3.28
23    2.82
24    3.47
25    5.20
26    4.59
dtype: float64